In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import typhon
import conrad
from netCDF4 import Dataset


plt.style.use(typhon.plots.styles('typhon'))

In [ ]:
def linear_profile(z, Ts=300, lrate=0.0065):
    return Ts - lrate * z

In [ ]:
data = Dataset('results/tropical-re.nc')

timestep = -1 

time = data.variables['time'][:] / 24  # time in days
p = data.variables['plev'][:]
ph = data.variables['phlev'][:]
z = data.variables['z'][timestep, :]
T = data.variables['T'][timestep, :]
dT = data.variables['deltaT'][timestep, :]

net_htngrt = data.variables['net_htngrt'][timestep, :]
lw_flxu = data.variables['lw_flxu'][timestep, :]
lw_flxd = data.variables['lw_flxd'][timestep, :]
sw_flxu = data.variables['sw_flxu'][timestep, :]
sw_flxd = data.variables['sw_flxd'][timestep, :]

fig, [ax, ax2] = plt.subplots(ncols=2, sharey=True, figsize=(8, 8))
fig.subplots_adjust(wspace=0.05)
typhon.plots.profile_p_log(p, T, label='Temperature', ax=ax, linewidth=4)
typhon.plots.profile_p_log(p, linear_profile(z, T[0]),
                           color='grey',
                           linestyle='dashed',
                           label='Const. lapse rate',
                           ax=ax,
                           )
ax.set_ylim(p.max(), p.min())
ax.set_xlim(180, 340)
ax.set_xlabel('Temperature [K]')
ax.grid(True)
ax.legend()

typhon.plots.profile_p_log(ph, lw_flxu, label='LW \u2191', lw=4, ax=ax2)
typhon.plots.profile_p_log(ph, lw_flxd, label='LW \u2193', lw=4, ax=ax2)
typhon.plots.profile_p_log(ph, sw_flxu, label='SW \u2191', lw=4, ax=ax2)
typhon.plots.profile_p_log(ph, sw_flxd, label='SW \u2193', lw=4, ax=ax2)
ax2.set_xticks(range(0, 1001, 200))
ax2.grid(True )
ax2.set_xlabel('Flux [$Wm^{-2}$]')
ax2.legend(ncol=2)

fig.savefig('plots/fluxes.pdf')

In [ ]:
data = Dataset('results/tropical_co2_x4-re.nc')

time = data.variables['time'][:] / 24  # time in days
lw_flxu = data.variables['lw_flxu'][:, -1]
lw_flxd = data.variables['lw_flxd'][:, -1]
sw_flxu = data.variables['sw_flxu'][:, -1]
sw_flxd = data.variables['sw_flxd'][:, -1]

toa = ((sw_flxd + lw_flxd) - (sw_flxu + lw_flxu))

fig, ax = plt.subplots()
ax.plot(time, toa, label='Radiative', linewidth=4)
ax.set_ylabel('TOA [$Wm^{-2}$]')
ax.set_xlabel('Days')
ax.set_xlim(left=0)
ax.legend()
ax.grid(True)
fig.savefig('plots/toa.pdf')

#
plev = data.variables['plev'][:]
lw_flxu = data.variables['lw_flxu'][:]
lw_flxd = data.variables['lw_flxd'][:]
sw_flxu = data.variables['sw_flxu'][:]
sw_flxd = data.variables['sw_flxd'][:]

toa = ((sw_flxd + lw_flxd) - (sw_flxu + lw_flxu))

fig, ax = plt.subplots()
sm = ax.pcolormesh(time, plev, toa[:, :-1].T, cmap='seismic', rasterized=True)
cb = fig.colorbar(sm)
typhon.plots.center_colorbar(cb)
ax.set_yscale('log')
ax.yaxis.set_major_formatter(typhon.plots.HectoPascalLogFormatter())
ax.yaxis.set_minor_formatter(typhon.plots.HectoPascalLogFormatter())
ax.set_ylim(plev.max(), plev.min())
#ax.set_ylabel('TOA [$Wm^{-2}$]')
ax.set_ylabel('Pressure [hPa]')
ax.set_xlabel('Days')
ax.set_xlim(left=0)
ax.legend()
ax.grid(True)
fig.savefig('plots/fluxes.pdf')

# Surface temperature (Downward fluxes)

In [ ]:
data = Dataset('results/tropical-re.nc')
time = data.variables['time'][:] / 24  # time in days

lw_flxd = data.variables['lw_flxd'][:, 0]
sw_flxu = data.variables['sw_flxu'][:, 0]
sw_flxd = data.variables['sw_flxd'][:, 0]

Ts = ((sw_flxd - sw_flxu + lw_flxd) / 5.67e-8)**(0.25)

fig, ax = plt.subplots()
ax.plot(time, Ts, label='Radiative', linewidth=4)
ax.set_ylabel('Surface Temperature [K]')
ax.set_xlabel('Days')
ax.set_xlim(left=0)
ax.grid(True)
fig.savefig('plots/surface.pdf')

In [ ]:
data = Dataset('results/tropical-re.nc')

timestep = -1

time = data.variables['time'][:] / 24  # time in days
p = data.variables['plev'][:]
ph = data.variables['phlev'][:]
z = data.variables['z'][timestep, :]
T = data.variables['T'][timestep, :]
q = data.variables['H2O'][timestep, :]

lw_flxd = data.variables['lw_flxd'][timestep, 0]
sw_flxu = data.variables['sw_flxu'][timestep, 0]
sw_flxd = data.variables['sw_flxd'][timestep, 0]
Ts = ((sw_flxd - sw_flxu + lw_flxd) / 5.67e-8)**(0.25)

atm = conrad.atmosphere.AtmosphereConvective.from_netcdf('results/tropical-re.nc', -1)
ctop = atm.convective_top(0.0065)
atm.convective_adjustment(ctop, lapse=0.0065)

In [ ]:
fig, [ax, ax2] = plt.subplots(ncols=2, sharey=True, figsize=(6, 8))
fig.subplots_adjust(wspace=0.05)
l = typhon.plots.profile_p_log(p, T, label='Radiative', ax=ax, linewidth=4)
ax.plot(Ts, p.max(), color=l[0].get_color(), linestyle='none', marker='o', label='$T_s$')
ax.axhline(p[atm.cold_point_index], color=l[0].get_color(), alpha=0.5, label='Cold point')
ax.set_ylim(p.max(), p.min())
ax.set_xlim(200, 350)
ax.set_xlabel('Temperature [K]')
ax.legend(loc='upper left')
ax.grid(True)

typhon.plots.profile_p_log(p, q, label='Radiative', ax=ax2, linewidth=4)
typhon.plots.profile_p_log(p, q, label='Radiative', ax=ax2, linewidth=4)
ax2.legend(loc='upper left')
ax2.grid(True)

fig.savefig('plots/profile.pdf')